In [ ]:
if 'google.colab' in str(get_ipython()):
    import sys
    from google.colab import drive, output
    drive.mount('/content/drive')
    !pip install torchaudio
    !pip install wandb --upgrade
    # !wandb login
    # !pip install umap-learn
    output.clear()
    print("Running on colab")
    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Dataloader')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Models')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Modules')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/temporal')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Extra_and_Unused')
    root = "/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project"
else:
    print("Running local")
    root = "/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics"
    root_path = "media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/"

In [2]:
# from __future__ import print_function
# from __future__ import print_function
import os
import numpy as np

import datetime
import gc

from scipy import signal

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import make_grid
import torchaudio.transforms as audio_transform

from Zamuro_DataLoader import SoundscapeData
from Models import ConvAE as AE
from AE_training_functions import TestModel, TrainModel # For AE
# from PosAE_training_functions import posautoencoding_m1 as AE, TestModel, TrainModel # For PosAE

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = xm.xla_device()
print(device)

from datetime import timedelta

# wandb.login()

cuda


In [29]:
def featuring_autoencoders(dataset, date_format, model, len_features=None, save=True, identifier=None):
    training_loader = DataLoader(dataset, batch_size=1)
    iterator = iter(training_loader)
    testing = TestModel(model, iterator, device=torch.device("cuda"))

    training_recorder_list = []
    training_hour_list = []
    training_minute_list = []
    delete_samples = []
    training_path_samples = []
    training_samples_list_torch = []
    
    if len_features == None:
        len_features = len(training_loader)
    else:
        len_features = len_features
        
    batch = int(len_features*0.1)
    
    for id in range(len_features):
    #     if (id+1)%3 == 0:
    #         break
        if (id+1)% batch == 0:
            print(f"id: {id + 1} of {len_features}")
        try:
            originals, reconstructions, encodings, label, loss, path = testing.reconstruct()
        except:
            print(f"error id: {id}")
            delete_samples.append(id)
            continue

    #     encodings_size = encodings[0].shape
        encodings = encodings.to("cuda").detach()
        encodings = encodings.reshape(encodings.shape[0],
                                    encodings.shape[1]*encodings.shape[2]*encodings.shape[3])
        training_samples_list_torch.append(encodings)
        label["recorder"]=np.repeat(label["recorder"][0][0],5)
        training_recorder_list.append(label["recorder"])
        training_hour_list.append(label["hour"].reshape(label["hour"].shape[0]*label["hour"].shape[1]))
        training_minute_list.append(label["minute"].reshape(label["minute"].shape[0]*label["minute"].shape[1]))


        path = np.asarray(path, dtype="U32")
        path = np.repeat(path, 5)
        indexer = ["_1","_2","_3","_4","_5"]
        indexer = np.asarray(indexer)
        # path = path.astype("float64")
        for i in range(len(path)):
#             print(path[i] + identifier[i])
            path[i] = path[i] + indexer[i]
#             print(path[i]) 
        training_path_samples.append(path)
    
#     training_recorder_list = torch.cat(training_recorder_list,dim=0)
    training_hour_list = torch.cat(training_hour_list,dim=0)
    training_minute_list = torch.cat(training_minute_list,dim=0)
    training_samples_list_torch = torch.cat(training_samples_list_torch, dim=0)
    
    if save == True:
    
        if "filters" in dataset.kwargs.keys():
            for value in dataset.filters.values():
                date_format = f"{date_format}_{value}" 
        print(identifier)
        if identifier != None:
            date_format = f"{date_format}_{identifier}"
        else:
            pass

        torch.save(training_path_samples, f"temporal_zamuro/Features/AE_test_path_samples_{date_format}.pth")
        torch.save(training_samples_list_torch, f"temporal_zamuro/Features/AE_features_{date_format}.pth")
        training_labels_list = {"recorder": training_recorder_list, "hour": training_hour_list, "minute": training_minute_list}
        torch.save(training_labels_list, f"temporal_zamuro/Features/AE_labels_{date_format}.pth")
        torch.save(delete_samples, f"temporal_zamuro/Features/AE_test_corrupted_samples_list_{date_format}.pth")
        
    else:
        pass

In [30]:
model = "AE"
identifier = "batch_size_14_num_hiddens_64_"
day = 4
hour = 9
date_format = f"day_{day}_hour_{hour}"
model_name = f"{root}/Zamuro/temporal_zamuro/models/model_{model}_{identifier}_{date_format}_final.pth"
config = torch.load(f'{root}/Zamuro/temporal_zamuro/configs/config_{model}_{identifier}_{date_format}.pth', map_location=torch.device('cpu'))
model = AE(num_hiddens=config["num_hiddens"]).to(device)
model.load_state_dict(torch.load(f'{model_name}', map_location=torch.device('cpu')))

<All keys matched successfully>

In [31]:
import os
folders = os.listdir(f"/{root_path}")

In [36]:
# folders = ["Audios_Jaguas"]
for folder in folders:
    print(folder)
    filters = {"rain_FI": "NO"}
    dataset = SoundscapeData('media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/',
                         dataframe_path="Complementary_Files/zamuro_audios.csv",
                         audio_length=12, ext="wav",
                         win_length=1028, filters=filters)
    featuring_autoencoders(dataset, f"{date_format}_rain", model=model, len_features=10, save=True, identifier=f"{folder}")

RZUB10
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 1 of 10
id: 2 of 10
id: 3 of 10
id: 4 of 10
id: 5 of 10
id: 6 of 10
id: 7 of 10
id: 8 of 10
id: 9 of 10
id: 10 of 10
RZUB10
RZUD08
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 1 of 10
id: 2 of 10
id: 3 of 10
id: 4 of 10
id: 5 of 10
id: 6 of 10
id: 7 of 10
id: 8 of 10
id: 9 of 10
id: 10 of 10
RZUD08
RZUF04
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 1 of 10
id: 2 of 10
id: 3 of 10
id: 4 of 10
id: 5 of 10
id: 6 of 10
id: 7 of 10
id: 8 of 10
id: 9 of 10
id: 10 of 10
RZUF04
RZUA01
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 1 of 10
id: 2 of 10
id: 3 of 10
id: 4 of 10
id: 5 of 10
id: 6 of 10
id: 7 of 10
id: 8 of 10
id: 9 of 10
id: 10 of 10
RZUA01
RZUA02
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 1 of 10
id: 2 of 10
id: 3 of 10
id: 4 of 10
id: 5 of 10
id: 6 of 10
id: 7 of 10
id: 8 of 10
id: 9 of 10
id: 10 of 

id: 5 of 10
id: 6 of 10
id: 7 of 10
id: 8 of 10
id: 9 of 10
id: 10 of 10
RZUD05
RZUD06
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 1 of 10
id: 2 of 10
id: 3 of 10
id: 4 of 10
id: 5 of 10
id: 6 of 10
id: 7 of 10
id: 8 of 10
id: 9 of 10
id: 10 of 10
RZUD06
RZUD07
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 1 of 10
id: 2 of 10
id: 3 of 10
id: 4 of 10
id: 5 of 10
id: 6 of 10
id: 7 of 10
id: 8 of 10
id: 9 of 10
id: 10 of 10
RZUD07
RZUD09
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 1 of 10
id: 2 of 10
id: 3 of 10
id: 4 of 10
id: 5 of 10
id: 6 of 10
id: 7 of 10
id: 8 of 10
id: 9 of 10
id: 10 of 10
RZUD09
RZUD10
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 1 of 10
id: 2 of 10
id: 3 of 10
id: 4 of 10
id: 5 of 10
id: 6 of 10
id: 7 of 10
id: 8 of 10
id: 9 of 10
id: 10 of 10
RZUD10
RZUD11
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 1 of 10
id: 2 of 10
id: 3 of 10
id

id: 2 of 10
id: 3 of 10
id: 4 of 10
id: 5 of 10
id: 6 of 10
id: 7 of 10
id: 8 of 10
id: 9 of 10
id: 10 of 10
RZUG11
RZUG12
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 1 of 10
id: 2 of 10
id: 3 of 10
id: 4 of 10
id: 5 of 10
id: 6 of 10
id: 7 of 10
id: 8 of 10
id: 9 of 10
id: 10 of 10
RZUG12
RZUG13
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 1 of 10
id: 2 of 10
id: 3 of 10
id: 4 of 10
id: 5 of 10
id: 6 of 10
id: 7 of 10
id: 8 of 10
id: 9 of 10
id: 10 of 10
RZUG13
RZUH02
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 1 of 10
id: 2 of 10
id: 3 of 10
id: 4 of 10
id: 5 of 10
id: 6 of 10
id: 7 of 10
id: 8 of 10
id: 9 of 10
id: 10 of 10
RZUH02
RZUH03
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 1 of 10
id: 2 of 10
id: 3 of 10
id: 4 of 10
id: 5 of 10
id: 6 of 10
id: 7 of 10
id: 8 of 10
id: 9 of 10
id: 10 of 10
RZUH03
RZUH04
/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id

In [14]:
filters = {"rain_FI": "NO"}
dataset = SoundscapeData('media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/',
                     dataframe_path="Complementary_Files/zamuro_audios.csv",
                     audio_length=12, ext="wav",
                     win_length=1028, filters=filters)

len_features=None
save=True
identifier=None

training_loader = DataLoader(dataset, batch_size=1)
iterator = iter(training_loader)
testing = TestModel(model, iterator, device=torch.device("cuda"))

training_recorder_list = []
training_hour_list = []
training_minute_list = []
delete_samples = []
training_path_samples = []
training_samples_list_torch = []

if len_features == None:
        len_features = len(training_loader)
else:
    len_features = len_features
        
batch = int(len_features*0.1)

for id in range(len_features):
#     if (id+1)%3 == 0:
#         break
    if (id+1)% batch == 0:
        print(f"id: {id + 1} of {len_features}")
        
(originals, reconstructions, encodings, label, loss, path) = testing.reconstruct()
#         except:
#             print(f"error id: {id}")
#             delete_samples.append(id)
#             continue

    #     encodings_size = encodings[0].shape
encodings = encodings.to("cuda").detach()
encodings = encodings.reshape(encodings.shape[0],
                            encodings.shape[1]*encodings.shape[2]*encodings.shape[3])
training_samples_list_torch.append(encodings)
label["recorder"]=np.repeat(label["recorder"][0][0],5)
training_recorder_list.append(label["recorder"])
training_hour_list.append(label["hour"].reshape(label["hour"].shape[0]*label["hour"].shape[1]))
training_minute_list.append(label["minute"].reshape(label["minute"].shape[0]*label["minute"].shape[1]))



/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/
id: 6507 of 65075
id: 13014 of 65075
id: 19521 of 65075
id: 26028 of 65075
id: 32535 of 65075
id: 39042 of 65075
id: 45549 of 65075
id: 52056 of 65075
id: 58563 of 65075
id: 65070 of 65075


In [ ]:
model_type = "AE"
identifier = "batch_size_14_num_hiddens_64_"
day = 9
hour = 4
month = 6
folder = "AE_rain"
date_format = f"day_{day}_hour_{hour}"
y = torch.load(f"temporal_zamuro/Features/{folder}/AE_labels_{date_format}_rain_NO_.pth",  map_location=torch.device('cpu'))
X = torch.load(f"temporal_zamuro/Features/{folder}/AE_features_{date_format}_rain_NO_.pth",  map_location=torch.device('cpu'))
path = torch.load(f"temporal_zamuro/Features/{folder}/AE_test_path_samples_{date_format}_rain_NO_.pth",  map_location=torch.device('cpu'))
path_flat = [item for sublist in path for item in sublist]
path_flat = np.asarray(path_flat)

In [33]:
X.shape

NameError: name 'X' is not defined

In [ ]:
folder = "Audios_Jaguas"
dataset = SoundscapeData(root_path, dataframe_path=f"Complementary_Files/Audios_Jaguas/{folder}.csv",
                             audio_length=12, ext="wav",
                             win_length=1028)# filters=filters)
training_loader = DataLoader(dataset, batch_size=1)
iterator = iter(training_loader)
testing = TestModel(model, iterator, device=torch.device("cuda"))
originals, reconstructions, encodings, label, loss, path = testing.reconstruct()
encodings[0].shape

In [ ]:
prueba = []

In [ ]:
prueba[0]

In [ ]:
training_loader = DataLoader(dataset, batch_size=1)
iterator = iter(training_loader)
testing = TestModel(model, iterator, device=torch.device("cuda"))
originals, reconstructions, encodings, label, loss, path = testing.reconstruct()
encodings = encodings.to("cuda").detach()
encodings = encodings.reshape(encodings.shape[0],
                            encodings.shape[1]*encodings.shape[2]*encodings.shape[3])
prueba.append(encodings)